# Entrega 1 - Jugador de Damas Chinas

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
El objetivo de esta tarea fue construir un jugador de **damas chinas** utilizando técnicas de aprendizaje automático. En este escenario se implementó una función de evaluación que ayuda al jugador a elegir su próxima jugada. Dicha función se ajusta a través de la experiencia jugando contra sí mismo o contra un jugador que elige su próximo movimiento de forma aleatoria.

En términos formales, los parámetros del problema se reducen a lo siguiente:
- **Tarea *T*:** Jugar una partida de Damas Chinas
- **Experiencia *E*:** Jugar contra un jugador aleatorio / Jugar contra sí mismo
- **Performance *P*:** Proporción de partidas ganadas

### 1.2. Entrega
***
La entrega de esta tarea consta de dos grandes componentes:
- **Informe** en formato de Jupyter Notebook (este informe).
- **Programa** simulador de juego y entrenamiento (y código fuente comentado).

El objetivo del informe es centralizar la información relativa a la construcción del modelo así como los datos surgidos de las distintas pruebas realizadas. Como agregado opcional, se adjuntan algunos scripts para ayudar a la lectura o probar en tiempo real ciertas funcionalidades.

Por otra parte, el simulador ofrece una interfaz en consola que permite entrenar jugadores con múltiples configuraciones paramétricas, obtener datos y gráficas del entrenamiento y además poder enfrentarse a ellos luego de entrenados.

Además de la lectura del informe, puede resultar de utilidad entrenar a un jugador en el simulador y jugar contra él, ya que se pueden realizar observaciones interesantes sobre la forma de jugar del mismo. Cabe destacar que dichas observaciones son de caracter empírico y si bien se realizan varias a lo largo del informe, no se toman en cuenta para sacar conclusiones determinantes.

### 1.3. Formato
***
En las siguientes secciones se especifica el diseño del modelo, justificando la toma de decisiones a la hora de construirlo. Luego, se muestran los resultados de probar distintos diseños para comparar sus rendimientos. Finalmente se agregan algunas conclusiones respecto a dichos rendimientos y sus respectivas justificaciones.

## 2. Diseño
***
En esta sección se detallan las características del diseño utilizado para construir el modelo.
Es importante tener en cuenta que esta etapa se vió afectada por la etapa de experimentación, lo que llevó a cambios de estrategia luego de encontrar problemas en el modelo. En la sección actual se hace enfasis en las opciones elegidas, mientras que en la sección de experimentación se toma un enfoque cronológico de los distintos enfoques tomados y como los resultados de las pruebas fueron influyendo en el cambio de diseño.

### 2.1. Juego
***

El juego **Damas Chinas** cuenta con múltiples variantes y reglas distintas. A continuación se especifican las reglas de su forma básica, así como los cambios implementados en este escenario con el motivo de simplificar u optimizar la resolución del problema:

**Jugadores:**
> - Puede haber entre 2 y 6 jugadores, los cuales juegan todos contra todos.
> - Cada jugador cuenta con 10 piezas.
> - *Simplificación:* Se limita la participación a 2 jugadores, con el objetivo de simplificar el problema a representar.

**Tablero:**
> - El tablero tiene forma de estrella de 6 puntas, el cual cuenta con 121 casillas dispuestas en forma hexagonal.
> - Esto implica que cada celda cuenta con 6 celdas adyacentes, a diferencia de una grilla rectangular, donde cada celda tiene 9 celdas adyacentes.
> - Cada jugador puede mover sus piezas hacia las celdas adyacentes que no estén ocupadas, o saltar piezas (suyas o del oponente) siempre y cuando el salto sea siguiendo la recta que contiene la pieza que salta y la pieza saltada.
> - Cada jugador comienza con sus 10 piezas en una punta de la estrella.
> - *Simplificación:* Se limita el tablero a ser un rombo de 81 casillas, eliminando los 4 triángulos que se encuentran a los lados del tablero. Dicha simplificación se realiza ya que en un juego entre 2, la existencia de dichos triángulos no tiene relevancia y solo agrega complejidad al representar el tablero.

<div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroOriginal.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 1 - Tablero original </label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroSimple.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 2 - Tablero simplificado </label>
    </div>
</div>

**Objetivo:**
> - El objetivo de un jugador es depositar todas sus fichas en el triángulo opuesto a su triángulo de inicio.
> - El juego se termina cuando uno de los jugadores cumple dicho objetivo, no existiendo posibilidad de empate.
> - *Simplificación:* Debido a que es posible que un partido se extienda en el tiempo tanto como los jugadores lo deseen, se agrega la posibilidad de **empatar** una partida, declarandose dicho empate luego de pasada una cierta cantidad de turnos configurable (por defecto 100).

En resumen, se realizan **3 modificaciones** a las reglas básicas del juego:
- Se limita la cantidad de jugadores a 2.
- Se transforma el tablero de una estrella de 121 casillas a un rombo de 81 casillas.
- Se agrega la posibilidad de empatar si no se logra terminar el juego luego de cierta cantidad de turnos.

### 2.2. Tablero
***
Como ya se mencionó en la anterior sección, el tablero de **Damas Chinas** tiene una estructura particular. A continuación se trata el problema subdividiendo en varias secciones:


#### 2.2.1. Representación física y lógica
***
Al tratarse de un tablero con una _grilla hexagonal_, no existe ninguna estructura predeterminada en Python para almacenarlo y mantener el movimiento de fichas sencillo e intuitivo. De esta forma, se generaron dos formas de representar el tablero: una **lógica** (para mostrarlo y realizar cálculos) y una **física** (para almacenarlo).

La **representación lógica** consta de un rombo de 81 casillas dispuestas hexagonalmente (cada casilla tiene 6 adyacentes excepeto las de los extremos) las cuales cuentan con un par de _coordenadas axiales_ para representarlas. Dicho sistema de coordenadas cuenta con 2 ejes como un sistema cartesiano, con la diferencia de que dichos ejes no son perpendiculares. Para obtener una referencia visual, correr **script 1** al final de la sección.

La **representación física**, por otra parte, consta de una matriz cuadrada 9x9 de 81 casillas dispuestas las cuales cuentan con un par de _coordenadas cartesianas_ para representarlas (una matriz común y corriente). Existe una _biyección_ entre las casillas en la representación lógica y en la física. Dicha biyección permite obtener las coordenadas axiales de un par de coordenadas cartesianas, existiendo una función inversa que permite hacer lo contrario. La biyección se compone de dos transformaciones lineales consecutivas:

1. **Traslación del centro:** Partiendo de la matriz, la cual cuenta con el centro en (0,0) en una esquina, se traslada dicho centro a la celda (4,4) (el centro exacto de la matriz cuadrada).
2. **Agregado de los extremos:** Partiendo de la matriz con centro trasladado, se sustituyen todas las coordenadas y que sean mayores a 4 o menores a -4 restandoles o sumandoles 9 respectivamente.

Sea $f: R^2 \rightarrow R^2$ la expresión que representa a la biyección, con $f_1$ y $f_2$ las anteriores transformaciones:

- $f(x,y)=f_2(f_1(x,y))$

- $f_1(x,y)=(x-4, -y+4)$

- $f_2(x,y)=$
    - $(x,y)$ $\forall$ $(x,y)$ $:$ $|y| < 4$
    - $(x,y-9)$ $\forall$ $(x,y)$ $:$ $y \geq 4$
    - $(x,y+9)$ $\forall$ $(x,y)$ $:$ $y \leq -4$

Para obtener una referencia visual, correr **script 1** al final de la sección.

#### 2.2.2. Representación de atributos
***
Debido a que existen miles de posibles tableros dependiendo de los movimientos que hagan ambos jugadores, se decidió que el enfoque para definir que atributos representarían a un tablero dado sería un enfoque de alto nivel, es decir, definiendo lineamientos a seguir y obteniendo atributos que se acoplen a los mismos. También es importante mencionar que se investigaron publicaciones existentes en referencia a problemas como este o similares.

De esta forma, se determinaron **3 métricas** infalibles a tener en cuenta al momento de jugar una partida, las cuales deberían indicar que tan bien se esta jugando para un jugador _i_ dado, siendo las mismas:
1. **$A_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia el objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a mover sus piezas al extremo opuesto del tablero.
2. **$B_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la línea vertical que representa el centro del tablero.** El objetivo de esta métrica es incitar al jugador _i_ a no alejar sus piezas de la línea central vertical del tablero, ya que esto en general consume movimientos y desperdicia posibles cadenas de saltos.
3. **$C_i$** - **Suma de la cantidad de saltos posibles de cada pieza del jugador _i_.** El objetivo de esta métrica es incitar al jugador _i_ a generar cadenas de saltos para hacer avanzar sus piezas más rápidamente.

Después de múltiples pruebas (se expanden las mismas en la sección 3.2), se consideró apropiado agregar una **4ta métrica** para ayudar a ajustar la función de evaluación ante ciertos tableros con representación ambigua utilizando solamente las 3 métricas anteriores. Esta nueva métrica es:
4. **$D_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la casilla libre más cercana en el triángulo objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a dirigirse al triángulo opuesto y rellenarlo con sus fichas. Se considera que esta métrica no sólo no es redundante con **$A_i$** , sino que además la complementa.

Al momento de evaluar un tablero es importante tener en cuenta la situación del oponente, por lo que se llegó a dos posibles conjuntos finales de atributos:
- **Conjunto de 9 atributos** = $[1, A_1, B_1, C_1, D_1, A_2, B_2, C_2, D_2]$ - Engloba a las 8 métricas mencionadas anteriormente, agregando 1 para poder contar con un peso $w_0$ independiente en la función de evaluación.
- **Conjunto de 5 atributos** = $[1, A, B, C, D]$ - Engloba también a las 8 métricas mencionadas anteriormente, generando una agrupación para la misma métrica de distinto jugador.

Si bien a priori resultan conjuntos redundantes, la motivación detrás del segundo conjunto es generar una proporción entre la situación del jugador *i* y de su oponente _j_ , definiendo:
- **$A = A_j - A_i$** - Si el jugador *i* está más lejos de su objetivo que el oponente _j_, el atributo toma un valor negativo, penalizando al jugador.
- **$B = B_j - B_i$** - Si el jugador *i* está más lejos del centro que el oponente _j_, el atributo toma un valor negativo, penalizando al jugador.
- **$C = C_i - C_j$** - Si el oponente *j* tiene la posibilidad de realizar más saltos que el jugador _i_, el atributo toma un valor negativo, penalizando al jugador.
- **$D = D_j - D_i$** - Si el jugador *i* está más lejos de las fichas libres en su objetivo que el oponente _j_, el atributo toma un valor negativo, penalizando al jugador. 

En la sección de experimentación se comparan estos dos conjuntos, evaluando el rendimiento de cada uno.

Finalmente, es importante detallar que los atributos obtenidos se **normalizan** pasando a estar en el intervalo $[0..1]$. Dicha normalización se implementa utilizando la **norma euclídea** y siguiendo la siguiente fórmula, siendo $x$ el vector de atributos, $x_i$ la coordenada _i_-ésima del mismo, $z$ el vector de atributos normalizado y $z_i$ la coordenada _i_-ésima del mismo:

$$z : z_i = \frac{x_i}{||x||}$$  

$$\forall i \in [0..length(x)]$$

#### 2.2.3. Script 1
***
A continuación se adjunta un script que permite comparar las distintas representaciones del tablero interactivamente, extrayendo sus atributos y viendo como el mover piezas afecta a los mismos.

In [1]:
# Script 1 - Representación del tablero
import ntShowBoard

Box(children=(Label(value='Representación del tablero:', layout=Layout(margin='0px 16px 0px 0px')), ToggleButt…


                                (+4 O -8)

                            (+3 O -7)(+4 O -7)

                        (+2 O -6)(+3 O -6)(+4 O -6)

                    (+1 O -5)(+2 O -5)(+3 O -5)(+4 O -5)

                (+0 X -4)(+1 X -4)(+2 X -4)(+3 X -4)(+4 X -4)

            (-1 X -3)(+0 X -3)(+1 X -3)(+2 X -3)(+3 X -3)(+4 X -3)

        (-2 X -2)(-1 X -2)(+0 X -2)(+1 X -2)(+2 X -2)(+3 X -2)(+4 X -2)

    (-3 X -1)(-2 X -1)(-1 X -1)(+0 X -1)(+1 X -1)(+2 X -1)(+3 X -1)(+4 X -1)

(-4 X +0)(-3 X +0)(-2 X +0)(-1 X +0)(+0 X +0)(+1 X +0)(+2 X +0)(+3 X +0)(+4 X +0)

    (-4 X +1)(-3 X +1)(-2 X +1)(-1 X +1)(+0 X +1)(+1 X +1)(+2 X +1)(+3 X +1)

        (-4 X +2)(-3 X +2)(-2 X +2)(-1 X +2)(+0 X +2)(+1 X +2)(+2 X +2)

            (-4 X +3)(-3 X +3)(-2 X +3)(-1 X +3)(+0 X +3)(+1 X +3)

                (-4 X +4)(-3 X +4)(-2 X +4)(-1 X +4)(+0 X +4)

                    (-4 O +5)(-3 O +5)(-2 O +5)(-1 O +5)

                        (-4 O +6)(-3 O +6)(-2 O +6)

                            (-4 O +7)(-

Box(children=(Box(children=(Label(value='Movimiento de jugador 1:', layout=Layout(margin='0px 16px 0px 0px')),…

### 2.3. Algoritmo
***
El algoritmo de evaluación y aprendizaje sigue las pautas generales del visto en el teórico. Se toman en cuenta las siguientes especificaciones:

#### 2.3.1. Función de valoración
***
El jugador a entrenar cuenta con una **función de valoración**, la cual le permite evaluar un tablero dado y determinar que tan bueno es para sí mismo. La representación de la misma es la siguiente:

$$f: T \rightarrow [-1,1] : f(x) = x * w $$

Siendo $T$ el espacio de los posibles tableros y $x$ la representación vectorial de los atributos de un tablero (pudiendo ser cualquiera de las 2 mencionadas en la sección 2.2.2) y siendo $w$ un vector de pesos con la siguiente estructura (dependiendo del conjunto de atributos elegido):

- **Vector de 9 pesos** = $[w_0, w_{A_1}, w_{B_1}, w_{C_1}, w_{D_1}, w_{A_2}, w_{B_2}, w_{C_2}, w_{D_2}]$
- **Vector de 5 pesos** = $[w_0, w_{A}, w_{B}, w_{C}, w_{D}]$

Es importante mencionar que la función de valoración devuelve 1 para "tableros ganadores", -1 para "tableros perdedores" y 0 para "tableros donde hay empate". En la sección de experimentación se mencionan ciertos ajustes realizados en relación a esto último, debido a problemas encontrados durante el entrenamiento.

#### 2.3.2. Ajuste de pesos
***
El ajuste del vector de pesos se realiza **al finalizar una partida**, iterando sobre el conjunto de tableros generados en la misma y actualizando los pesos por cada par $(t, f'(t))$ siendo $t$ cada tablero y $f'$ la función de valoración utilizada en dicha partida.

Sea $w$ el vector de pesos, $w_i$ la coordenada _i_-ésima del mismo, $f(t)$ la valoración del tablero $t$ con la función actual y $f'(t)$ la valoración del siguiente tablero realizada durante el juego, la regla de actualización de pesos es la siguiente: 

$$w : w_i = w_i + (f'(t) - f(t))^2$$

$$\forall i \in [0..length(w)]$$

Dicha fórmula se basa en ajustar los pesos siguiendo el **error cuadrático medio** entre la valoración previa y la actual, tomando en cuenta que la valoración actual ajusta a más tableros y por lo tanto "tiene más experiencia".

Es importante destacar que de la forma en la que están pensados los atributos (para el conjunto de 5) se sabe que los pesos $w_{A}, w_{B}, w_{C}, w_{D}$ deben ser todos positivos ya que en caso contrario estarían fomentando acciones que atentan en contra la victoria del jugador y no a su favor (se expande esto en la sección 3.2). Para evitar este problema, es posible aplicar diversas técnicas de **normalización** (se expande esto en las secciones 3.2 y 3.3).

Para el caso donde los atributos se representan con un vector de 5, se aplicó la normalización bajo la **norma min-max** que utiliza la siguiente fórmula, siendo $w$ el vector de pesos, $w_i$ la coordenada _i_-ésima del mismo, $z$ el vector de pesos normalizado y $z_i$ la coordenada _i_-ésima del mismo:

$$z : z_i = \frac{w_i - min(w)}{max(w) - min(w)}$$  

$$\forall i \in [0..length(w)]$$

De esta manera, se restringe el valor de los pesos al intervalo $[0..1]$, siendo siempre positivos y por lo tanto no atentando contra las decisiones del jugador.

Para el caso donde los atributos se representan con un vector de 9, no se aplicó normalización ya que los pesos relativos a los atributos del jugador deberían ser positivos siempre y los pesos relativos a los atributos del oponente deberían ser negativos siempre. No obstante, no se descartó que el algoritmo de ajuste pudiera generar pesos del signo opuesto al esperado (se expande esto en la sección 3.3). 

#### 2.3.3. Ratio de aprendizaje y enfriamiento
***
Es posible agregar una constante $\gamma$ denominada **ratio de aprendizaje** que influye directamente sobre el ajuste de los pesos. La misma lo hace siguiendo la fórmula:

$$w : w_i = w_i + \gamma (f'(t) - f(t))^2$$

$$\forall i \in [0..length(w)]$$

En resumen, se multiplica el error cuadrático medio por el ratio de aprendizaje. Un mayor valor de $\gamma$ implicaría ajustar la función más rápido, mientras que un valor más pequeño lo haría más lento. En la sección 3.3 se comparan distintos valores de $\gamma$ y su influencia en el rendimiento de cada modelo.

Además de implementar a $\gamma$ como una constante, también se puede hacer que sea *variable*, disminuyendo su valor a medida que el entrenamiento avanza. Esto se denomina **enfriamiento** y se hace con el fin de no sobreajustar la función o no oscilar alrededor del valor de convergencia. En la sección 3.3 se utiliza un enfriamiento de valor inicial 0.9 y valor final 0.1, disminuyendo en 0.1 cada 10 iteraciones. 


### 2.4. Oponentes
***
Se entrenó contra dos tipos bien diferenciados de oponentes:

#### 2.4.1. Jugador aleatorio
***
**Funcionamiento:** Se define como **jugador aleatorio** a un programa que elige sus jugadas de forma aleatoria. Utilizando la biblioteca `random` de Python, el jugador aleatorio evalúa todos los posibles movimientos para cada una de sus fichas y elige uno de los mismos al azar.

**Motivo:** Puede resultar interesante evaluar la forma en la que el jugador aprende a entrenar al tener un oponente que no juega intentando ganar.

**Consideraciones al entrenar:** Debido a que las reglas del juego establecen que una partida se termina cuando un oponente rellena el triángulo opuesto con sus fichas, resulta muy probable que el jugador aleatorio no libere todo el triángulo y el tiempo de entrenamiento tienda a infinito, afectando a los resultados y a los ajustes (se expande este punto en la sección 3.5).

#### 2.4.2. Jugador previo
***
**Funcionamiento:** Se define como **jugador previo** a un programa que elige sus jugadas utilizando la misma función de evaluación que uso su oponente en el partido anterior. En otras palabras, es la versión del jugador que se está entrenando pero en la anterior iteración. 
Sin entrar en detalle, evalúa todos los tableros resultantes de todos los posibles movimientos para cada una de sus fichas y elige uno de la lista de movimientos con mejor evaluación. Dicha elección puede hacerse de forma aleatoria o elegir el primer movimiento que se calcule (esto puede traer problemas, se expande este punto en la sección 3.2).

**Motivo:** Puede ser una buena forma de automatizar el aprendizaje del juego contra un oponente que intenta ganar de la misma forma que el jugador a entrenar.

**Consideraciones al entrenar:** Existen distintas etapas donde es posible actualizar los pesos de la función objetivo del oponente. Para este escenario, los pesos del jugador a entrenar se actualizan al final de cada partida, tomando en cuenta todos los tableros generados en la misma. Los pesos del oponente se copian de los pesos del jugador también al final de cada partida, antes de ajustarlos.

## 3. Experimentación
***
En esta sección se detalla el conjunto de pruebas realizadas, agregandose observaciones pertinentes y comparaciones entre modelos.
A diferencia de la etapa anterior, se hace hincapié en como las pruebas influyeron en el proceso de construcción del modelo. Finalmente, luego de tener un diseño aparentemente libre de errores conceptuales, se comparan los rendimientos de distintas configuraciones paramétricas. 

### 3.1. Metodología
***

Con el objetivo de encontrar un modelo con una relativa eficiencia a la hora de aprender y jugar, se encaró la exprimentación diviendola en las siguientes etapas:

1. **Recopilación y ajuste:** Debido a las diversas opciones a la hora de realizar el diseño del modelo, se tomó un enfoque de ensayo y error; para cada posible decisión se realizaron pruebas y en base a los resultados se tomó o descartó la decisión. En esta etapa se detalla el proceso de construcción.
<br><br>
2. **Configuraciones paramétricas:** Luego de llegar a un modelo relativamente libre de errores (al menos de errores identificados), se realizaron pruebas para múltiples configuraciones paramétricas y comparar cuales de ellas resultaron ser las mejores. En esta etapa se detallan los datos obtenidos.
<br><br>
3. **Elección y comparación de representantes:** A falta de suficientes corridas para generar agrupamientos de configuraciones (como las obtenidas en el *Test de Friedman*), se compararon manualmente los resultados obtenidos en las pruebas de cada configuración paramétrica de la etapa anterior, eligiendo aquellas que optimizaron el ratio de partidas ganadas sobre el total de jugadas.
<br><br>
4. **Entrenamiento contra jugador aleatorio:** Debido a las características especiales del juego se separó en otra etapa el entrenamiento contra un jugador aleatorio, detallando los desafíos encontrados y los datos obtenidos a través de la experimentación

### 3.2. Recopilación y ajuste
***

Esta etapa se denomina **Recopilación y ajuste** ya que se fue ajustando el diseño del modelo en base a una recopilación importante de datos. Dicha recopilación se realizó en fases estructuradas, consistiendo las mismas de las siguientes acciones:

1. **Definición de las caracteristicas del modelo:** Determinar características como el conjunto de atributos para representar un tablero, la lógica a la hora de elegir las mejores jugadas o determinar un empate, ajustes en el modelo matemático, etc.
<br><br>
2. **Generación de datos de prueba y rendimiento:** Entrenar el modelo utilizando distintas configuraciones paramétricas y extraer datos básicos del entrenamiento como cantidad de partidas ganadas, perdidas o empatadas, como se distribuyen esos resultados en el tiempo, etc.
<br><br>
3. **Observación y detección de problemas:** En base a los resultados obtenidos del entrenamiento, monitorearlo y jugar contra el jugador entrenado con el objetivo de encontrar problemas específicos y relacionarlos al diseño del modelo.
<br><br>
4. **Adaptación del modelo:** En base a los problemas detectados, determinar las nuevas estrategias a implementar.

A continuación se detalla la información correspondiente a cada fase:

#### 3.2.1. Fase 1 - Modelo básico
***
**Definición:** Se utilizaron las siguientes especificaciones:
- *Atributos:* **$1$**, **$A$**, **$B$**, **$C$**
- *Elección:* **No aleatoria**
- *Empate:* **Si**
- *Normalización:* **No**

**Pruebas y resultados:**
- Se entrenaron distintas instancias del modelo, alternando el **ratio de aprendizaje** entre 0.5 y 0.9 y los **pesos iniciales** entre $[0.9, 0.9, 0.9, 0.9]$, $[0.5, 0.5, 0.5, 0.5]$ y $[0.1, 0.9, 0.1, 0.1]$.
- Los resultados de dicho entrenamiento fueron independientes a los ajustes paramétricos: después de un par de partidas ganadas o perdidas, todos los jugadores entraban en una racha de empates hasta el final del entrenamiento (100 iteraciones). 

**Problemas detectados:**
- Se monitorearon las decisiones tomadas por el jugador durante el entrenamiento y se comprobó que en cada racha de empates sucedía lo mismo: el jugador y su oponente en vez de completar el triángulo opuesto repetían el mismo movimiento una y otra vez alrededor del eje central. No se logró identificar con exactitud la razón de esto pero se diseñaron dos estrategias para contrarrestarlo.
- Por un lado se implementó la métrica $D$, con el objetivo de que el jugador priorizara el avance hacia las casillas vacías del triángulo opuesto.
- Por otra parte se implementó una elección aleatoria del conjunto de mejores tableros encontrados. En esta versión siempre se elegia el primero, por lo que toda simulación para determinados pesos daba siempre el mismo resultado.

**Cambios para la siguiente fase:**
- Se implementaron las estrategias mencionadas en el punto anterior.

#### 3.2.2. Fase 2 - Implementación de $D$ y elección aleatoria
***
**Definición:** Se utilizaron las siguientes especificaciones:
- *Atributos:* **$1$**, **$A$**, **$B$**, **$C$**, **$D$**
- *Elección:* **Aleatoria**
- *Empate:* **Si**
- *Normalización:* **No**

**Pruebas y resultados:**
- Se utilizaron las mismas configuraciones paramétricas que en el punto anterior
- La implementación de la nueva métrica D y del algoritmo de elección aleatoria ayudó a disminuir la producción de empates, sin embargo, estos eventualmente ocurrieron, causando la misma cadena.

**Problemas detectados:**
- Se monitorearon las decisiones tomadas por el jugador durante el entrenamiento y se comprobó que los primeros empates causaron ruido en el ajuste, penalizando un tablero en donde el jugador hizo bien sus jugadas.
- Esto causó que los pesos se ajusten a valores negativos, y el jugador buscara maximizar su distancia a la meta en lugar de minimizar la misma, generando de nuevo bucles de jugadas.

**Cambios para la siguiente fase:**
- Con el fin de estudiar como se comportaría el jugador si los pesos fueran todos positivos, se implementó que luego del ajuste se sustituya cada peso por su valor absoluto


#### 3.2.3. Fase 3 - Implementación de valor absoluto
***
**Definición:** Se utilizaron las siguientes especificaciones:
- *Atributos:* **$1$**, **$A$**, **$B$**, **$C$**, **$D$**
- *Elección:* **Aleatoria**
- *Empate:* **Si**
- *Normalización:* **Valor absoluto**

**Pruebas y resultados:**
- Se utilizaron las mismas configuraciones paramétricas que en el punto anterior. Se agregó la implementación de un ratio de aprendizaje variable con enfriamiento.
- Los resultados fueron mucho mejores que en las fases anteriores: se empató menos de 20 partidas en cada entrenamiento y se ganó y perdió en cantidades similares.

**Problemas detectados:**
- La implementación del valor absoluto al ajustar hizo que desaparecieran los bucles de jugadas, pero al monitorear las decisiones de un jugador entrenado se encontró que tomaba decisiones extrañas en diversos momentos, como alejarse del centro o romper una cadena de saltos sin motivo aparente.
- Después de un análisis más exhaustivo, se determinó que el haber implementado valor absoluto llevó a que valores negativos más pequeños (por ejemplo, -0.9) se priorizaran al ser transformados a positivos por sobre otros valores ya positivos (por ejemplo, 0.5), afectando en la toma de decisiones.

**Cambios para la siguiente fase:**
- Si bien dió mejores resultados en términos estadísticos, el valor absoluto era un parche de prueba y por lo tanto se eliminó
- Además, se implementó la eliminación de la valoración de tableros de empate. En la fase anterior se determinó empíricamente que luego del primer empate, el jugador comenzaba a ajustar sus pesos de manera extraña. Esto puede deberse a que la valoración de empate de ciertos tableros no tenía que ver con los tableros en sí, sino con haber jugado cierta cantidad de turnos. Esta estrategia significó ruido a la hora de valorar, por lo que fue eliminada, salteandose la etapa de ajuste en el caso de empatar.

#### 3.2.4. Fase 4 - Eliminación de empate
***
**Definición:** Se utilizaron las siguientes especificaciones:
- *Atributos:* **$1$**, **$A$**, **$B$**, **$C$**, **$D$**
- *Elección:* **Aleatoria**
- *Empate:* **No**
- *Normalización:* **No**

**Pruebas y resultados:**
- Se utilizaron las mismas configuraciones paramétricas que en el punto anterior.
- Los resultados fueron mejores que en las fase 2 pero se volvió a tener bucles de empate en las últimas iteraciones.

**Problemas detectados:**
- El mismo problema que en la fase 2: si bien la eliminación de ajuste luego de empate mejoró notablemente el aprendizaje, los bucles volvieron aunque un poco más adelante. Al haber pesos negativos se volvió a penalizar las acciones que harían ganar al jugador, volviendo a empatar indefinidamente. 
 
**Cambios para la siguiente fase:**
- Se generaron 2 modelos con el objetivo de palear el problema de pesos negativos. En la próxima sección se detallan.

#### 3.2.5. Fase 5 - Modelos finales
***
**Definición:** Se utilizaron las siguientes especificaciones:
- *Atributos:* $[1, A, B, C, D]$ y $[1, A_1, B_1, C_1, D_1, A_2, B_2, C_2, D_2]$
- *Elección:* **Aleatoria**
- *Empate:* **No**
- *Normalización:* **Sí** con 5 atributos, **No** con 9 atributos

Estas 2 instancias del modelo fueron utilizadas al realizar pruebas de las distintas configuraciones paramétricas, por lo que se estudian en detalle en la siguiente sección.

### 3.3. Configuraciones paramétricas
***

#### 3.3.1. Instancias
***
Como se menciona en la anterior sección, se generaron 2 instancias del modelo con el fin de comparar el rendimiento de cada una y determinar cual se ajusta mejor al problema. Dichas instancias se denominan **Instancia 1** e **Instancia 2**, contando cada una con las siguientes características:

1. **Instancia 1**
    - *Tablero de 5 atributos:* $[1, A, B, C, D]$
    - *Normalización de pesos:* **Sí** (norma mín-máx)
    - *Ajuste en empates:* **No**
<br><br>
2. **Instancia 2**
    - *Tablero de 9 atributos:* $[1, A_1, B_1, C_1, D_1, A_2, B_2, C_2, D_2]$
    - *Normalización de pesos:* **No**
    - *Ajuste en empates:* **No**

#### 3.3.2. Parámetros
***
Para cada instancia se eligieron múltiples configuraciones paramétricas, a modo de estudiar la influencia de distintos parámetros al momento de realizar el entrenamiento. Los parámetros que se variaron en dichas configuraciones son:

- **Ratio de Aprendizaje**
    - *Motivación:* Observar la incidencia del ajuste de los pesos en el rendimiento del jugador.
    - *Valores:*
        - **$0.5$**
        - **$0.9$**
        - **$0.1$**
        - **Enfriamiento** (de $0.9$ a $0.1$ disminuyendo $0.1$ cada $10$ iteraciones)
<br><br>
- **Pesos Iniciales**
    - *Motivación:* Observar la incidencia de priorizar ciertos pesos sobre otros en el rendimiento del jugador.
    - *Valores:* 
        - $[0.9, 0.9, 0.9, 0.9, 0.9]$
        - $[0.1, 0.9, 0.1, 0.1, 0.1]$
        - $[0.5, 0.5, 0.5, 0.5, 0.5]$

#### 3.3.3. Resultados
***
A continuación se adjuntan los resultados obtenidos en el entrenamiento de cada jugador siguiendo distintas configuraciones paramétricas. Dichos resultados se centralizan en una tabla, destacando aquellos que dieron un buen rendimiento y estudiandolos con mayor profundidad en la siguiente sección.

Además de las tablas para cada instancia, se adjuntan gráficas de los resultados a través del tiempo y el error cuadrático medio obtenido al ajustar los pesos en distintas iteraciones. Para aquellos modelos que brinden un rendimiento mayor que el resto, se analizarán dichas gráficas.

#### 3.3.3.1. Instancia 1
***
#### 3.3.3.1.1. Resultados de distintas configuraciones paramétricas
***

A continuación se presentan los resultados de todos los entrenamientos ejecutados para la **instancia 1**.
- Se marca en celeste las configuraciones paramétricas que dieron mejor resultado para cada ratio de aprendizaje utilizado.
- Se marca en verde la configuración paramétrica que dio el mejor resultado.

<table>
    <tr>
        <th>Iteraciones</th>
        <th>Turnos</th>
        <th>Ratio $\gamma$</th>
        <th>Pesos iniciales</th>
        <th>Pesos finales (aprox)</th>
        <th>Partidas ganadas</th>
        <th>Partidas perdidas</th>
        <th>Partidas empatadas</th>
        <th>Ratio de victorias</th>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.50, 0.00, 1.00, 0.20, 0.42]</td>
        <td>13%</td>
        <td>7%</td>
        <td>80%</td>
        <td>0,65</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.27, 0.00, 0.08, 0.91, 1.00]</td>
        <td>7%</td>
        <td>2%</td>
        <td>91%</td>
        <td>0,78</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.68, 0.24, 0.00, 1.00, 0.38]</td>
        <td>21%</td>
        <td>6%</td>
        <td>73%</td>
        <td>0,78</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 1.00, 0.76, 0.69, 0.60]</td>
        <td>35%</td>
        <td>14%</td>
        <td>51%</td>
        <td>0,71</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>58%</td>
        <td>24%</td>
        <td>18%</td>
        <td>0,70</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.83, 1.00, 0.65, 0.64, 0.00]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.00, 0.56, 1.00, 0.36, 0.29]</td>
        <td>48%</td>
        <td>34%</td>
        <td>18%</td>
        <td>0,58</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.20, 0.68, 1.00, 0.32, 0.00]</td>
        <td>44%</td>
        <td>32%</td>
        <td>24%</td>
        <td>0,57</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[0.21, 0.74, 0.96, 1.00, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.9, 0.9, 0.9, 0.9, 0.9]</td>
        <td>[0.39, 0.80, 1.00, 0.53, 0.00]</td>
        <td>50%</td>
        <td>28%</td>
        <td>22%</td>
        <td>0,64</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.1, 0.9, 0.1, 0.1, 0.1]</td>
        <td>[0.00, 0.39, 1.00, 0.53, 0.18]</td>
        <td>55%</td>
        <td>17%</td>
        <td>28%</td>
        <td>0,76</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.5, 0.5, 0.5, 0.5, 0.5]</td>
        <td>[1.00, 0.86, 0.96, 0.36, .00]</td>
        <td>31%</td>
        <td>6%</td>
        <td>63%</td>
        <td>0,83</td>
    </tr>
    <caption>Tabla 1 - Resultados de entrenamiento de <b>Instancia 1</b> para cada configuración paramétrica</caption>
</table>

#### 3.3.3.1.2. Resultados a lo largo de las iteraciones para distintas configuraciones paramétricas
***
A continuación se adjuntan las gráficas de resultados en función del tiempo para los 4 mejores modelos:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ResultsAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- Independiente al ratio de aprendizaje, las victorias y derrotas se distribuyen de una forma relativamente uniforme. Esto puede implicar que el algoritmo no esta ajustando de la mejor manera, ya que a más iteraciones, debería haber menos densidad de derrotas. No obstante, al ser sólo 100 iteraciones podría no significar nada.
- Para un ratio $\gamma = 0.5$ y un vector inicial de pesos $w = [0.5, 0.5, 0.5, 0.5, 0.5]$, puede verse que a partir de cerca de la mitad del entrenamiento el jugador deja de ganar o perder, empatando (al no contabilizarse dichos resultados para ajustar, no se grafican). Esto puede implicar que este jugador ajusta particularmente mal, generando bucles a partir de cierto momento.

#### 3.3.3.1.3. Error Cuadrático Medio cada 10 iteraciones para distintas configuraciones paramétricas
***

A continuación se adjuntan las gráficas de error cuadrático medio a medida que se ajustaron los pesos para los 4 mejores modelos. Cabe destacar que las curvas graficadas representan el ajuste cada 10 iteraciones, con el fin de estudiar la variación en el error a medida que se llevaba a cabo el entrenamiento:

<div style="margin-top: 16px; margin-bottom: 24px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_05_ErrorsAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/1_ErrorAll05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.9, pesos [0.5, 0.5, 0.5, 0.5, 0.5]</label>
    </div>
    <div style="display: inline-block; width:24%; text-align: center;">
        <img src="img/1_ErrorsAll09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.1, pesos [0.9, 0.9, 0.9, 0.9, 0.9]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
	<img src="img/1_ErrorAll01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio variable, pesos [0.1, 0.9, 0.1, 0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- Independiente al ratio de aprendizaje, se puede ver que el error cuadrático medio (ecm) oscila más bruscamente en las primeras iteraciones.
- Para un ratio $\gamma = 0.1$ y un vector inicial de pesos $w = [0.9, 0.9, 0.9, 0.9, 0.9]$, esto último se cumple particularmente. A priori podría interpretarse como que el jugador está convergiendo a una buena función de evaluación, no obstante, si se comparan los resultados, el rendimiento del jugador entrenado con esta confgiuración es pobre en comparación a los otros, teniendo un no muy buen ratio de victorias.
- Para un ratio $\gamma = 0.5$ y un vector inicial de pesos $w = [0.5, 0.5, 0.5, 0.5, 0.5]$, puede verse que los pesos se ajustan mucho menos mostrando solo 3 iteraciones (lo que equivaldría a alrededor de 30 iteraciones de ajuste). Esto se debe a la gran cantidad de empates para esta configuración.

#### 3.3.3.2. Instancia 2
***
#### 3.3.3.2.1. Resultados de distintas configuraciones paramétricas
***
A continuación se presentan los resultados de todos los entrenamientos ejecutados para la **instancia 2**.
- Se marca en celeste las configuraciones paramétricas que dieron mejor resultado para cada ratio de aprendizaje utilizado.
- Se marca en verde la configuración paramétrica que dio el mejor resultado.

<table>
    <tr>
        <th>Iteraciones</th>
        <th>Turnos</th>
        <th>Ratio $\gamma$</th>
        <th>Pesos iniciales</th>
        <th>Pesos finales (aprox)</th>
        <th>Partidas ganadas</th>
        <th>Partidas perdidas</th>
        <th>Partidas empatadas</th>
        <th>Ratio de victorias</th>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.9, -0.9, 0.9, -0.9, 0.9, 0.9, -0.9, -0.9, 0.9]</td>
        <td>[0.90, 0.25, 0.89, -0.85, 0.92, 0.99, -0.90, 0.25, 0.69]</td>
        <td>28%</td>
        <td>0%</td>
        <td>72%</td>
        <td>1,0</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</td>
        <td>[0.09, -0.34, 1.27, -0.24, 0.17, 0.16, 0.10, 0.45, 0.32]</td>
        <td>78%</td>
        <td>22%</td>
        <td>0%</td>
        <td>0,78</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.5</td>
        <td>[0.5, -0.5, 0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5]</td>
        <td>[0.50, 0.40, 0.75, -0.45, 0.53, 0.53, -0.48, 0.40, 0.40]</td>
        <td>39%</td>
        <td>0%</td>
        <td>61%</td>
        <td>1,0</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.9, -0.9, 0.9, -0.9, 0.9, 0.9, -0.9, -0.9, 0.9]</td>
        <td>[0.91, -0.23, 1.52, -0.85, 0.95, 0.90, -0.85, -0.23, 1.11]</td>
        <td>2%</td>
        <td>0%</td>
        <td>98%</td>
        <td>1,0</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</td>
        <td>[0.10, -0.22, 1.63, -0.20, 0.23, 0.13, 0.01, 0.57, -0.16]</td>
        <td>51%</td>
        <td>17%</td>
        <td>32%</td>
        <td>0,75</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.9</td>
        <td>[0.5, -0.5, 0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5]</td>
        <td>[0.50, 0.38, 0.78, -0.45, 0.53, 0.54, -0.49, 0.38, 0.41]</td>
        <td>28%</td>
        <td>0%</td>
        <td>72%</td>
        <td>1,0</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.9, -0.9, 0.9, -0.9, 0.9, 0.9, -0.9, -0.9, 0.9]</td>
        <td>[0.90, 0.21, 1.13, -0.86, 0.94, 1.00, -0.88, 0.21, 0.83]</td>
        <td>39%</td>
        <td>0%</td>
        <td>61%</td>
        <td>1,0</td>
    </tr>
    <tr style="background-color: #95dcd4;">
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</td>
        <td>[0.098, -1.22, 1.30, -0.14, 0.13, 0.04, -0.02, -0.42, -0.01]</td>
        <td>86%</td>
        <td>14%</td>
        <td>0%</td>
        <td>0,86</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>0.1</td>
        <td>[0.5, -0.5, 0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5]</td>
        <td>[0.50, -0.09, 0.88, -0.47, 0.53, 0.51, -0.47, -0.09, 0.63]</td>
        <td>4%</td>
        <td>0%</td>
        <td>96%</td>
        <td>1,0</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.9, -0.9, 0.9, -0.9, 0.9, 0.9, -0.9, -0.9, 0.9]</td>
        <td>[0.91, -0.22, 1.51, -0.85, 0.95, 0.91, -0.85, -0.22, 1.10]</td>
        <td>2%</td>
        <td>0%</td>
        <td>98%</td>
        <td>1,0</td>
    </tr>
    <tr style="background-color: #afdb99;">
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</td>
        <td>[0.10, -0.22, 1.25, -0.19, 0.07, 0.07, 0.04, 0.57, 0.13]</td>
        <td>89%</td>
        <td>11%</td>
        <td>0%</td>
        <td>0,89</td>
    </tr>
    <tr>
        <td>100</td>
        <td>100</td>
        <td>Enfriamiento</td>
        <td>[0.5, -0.5, 0.5, -0.5, 0.5, 0.5, -0.5, -0.5, 0.5]</td>
        <td>[0.91, -0.22, 1.51, -0.85, 0.95, 0.91, -0.85, -0.22, 1.10]</td>
        <td>37%</td>
        <td>0%</td>
        <td>63%</td>
        <td>1,0</td>
    </tr>
    <caption>Tabla 2 - Resultados de entrenamiento de <b>Instancia 2</b> para cada configuración paramétrica</caption>
</table>

#### 3.3.3.2.2. Resultados a lo largo de las iteraciones para distintas configuraciones paramétricas
***

A continuación se adjuntan las gráficas de resultados en función del tiempo para los 4 mejores modelos:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_Graph05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.3 - Entrenamiento con ratio 0.5, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_Graph09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.1 - Entrenamiento con ratio 0.9, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1] </label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_Graph01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.2 - Entrenamiento con ratio 0.1, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1] Ratio 0.1 </label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_Graphvar.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.4 - Entrenamiento con ratio variable, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- Para un ratio $\gamma = 0.1$ y un vector inicial de pesos $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, se puede observar que las derrotas, si bien son pocas, se encuentran agrupadas cerca del final del entrenamiento. Esto podría implicar que el modelo fue empeorando su aprendizaje con el correr de las iteraciones. No obstante, la cantidad de iteraciones no es suficiente como para determinar una relación. Además, el modelo resultante tuvo el segundo mejor rendimiento de todos los entrenados, por lo que en un principio no habría relación.
- Para todos los demás modelos puede verse que las derrotas se agrupan al principio, pudiendo indicar un mejor desempeño en el aprendizaje. Particularmente el mejor modelo (ratio variable) cuenta solamente con 3 derrotas en el medio del entrenamiento.

#### 3.3.3.2.3. Error Cuadrático Medio cada 10 iteraciones para distintas configuraciones paramétricas
***

A continuación se adjuntan las gráficas de error cuadrático medio a medida que se ajustaron los pesos para los 4 mejores modelos. Cabe destacar que las curvas graficadas representan el ajuste cada 10 iteraciones, con el fin de estudiar la variación en el error a medida que se llevaba a cabo el entrenamiento:

<div style="margin-top: 16px; margin-bottom: 24px;">
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_MSE05.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.7 - Entrenamiento con ratio 0.5, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
    <div style="display: inline-block; width:24%; text-align: center;">
        <img src="img/2_0_1_MSE09.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.5 - Entrenamiento con ratio 0.9, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_MSE01.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.6 - Entrenamiento con ratio 0.1, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
    <div style="display: inline-block; width: 24%; text-align: center;">
        <img src="img/2_0_1_MSEvar.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.8 - Entrenamiento con ratio variable, pesos [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]</label>
    </div>
</div>

**Observaciones:**
- Independiente al ratio de aprendizaje, se puede ver que el error cuadrático medio (ecm) oscila más bruscamente en las primeras iteraciones, obteniendo picos sobre el final del juego.
- Para un ratio $\gamma = 0.9$ y un vector inicial de pesos $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, puede verse que oscila en un rango menor al resto, sólo comparandose con el modelo entrenado con ratio variable. Es difícil interpretar que implicaría esto, ya que el primer jugador es el de peor rendimiento de los 4 mientras que el segundo es el de mejor rendimiento.

### 3.4. Elección y comparación de representantes
***

#### 3.4.1. Métricas
***
Habiendo entrenado 24 jugadores distintos, resulta pertinente determinar de la forma menos errónea posible, cuales de ellos tuvieron un **mejor** rendimiento. Se utilizan dos métricas distintas con este fin:
- **Cantidad de Partidas Ganadas (_win rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas jugadas independientemente a su resultado, siguiendo la fórmula $$winRate = \frac{ganadas}{jugadas}$$
<br><br>
- **Ratio de Victorias (_victory rate_)**: Representa la cantidad de partidas ganadas sobre el total de partidas con algún resultado, es decir, no empatadas, siguiendo la fórmula $$victoryRate = \frac{ganadas}{ganadas + perdidas}$$
<br><br>
El motivo de tomar la segunda métrica además de la primera, es el hecho de que un jugador con más victorias que otro no es necesariamente mejor. Sea el escenario donde un **jugador 1** gana y pierde 50 partidas respectivamente y un **jugador 2** gana 25 y empata 75. Si bien **jugador 1** ganó el doble de partidas que **jugador 2**, perdió 50 partidas más. El motivo de los empates de **jugador 2** es indeterminado, por lo que no se puede decir quien de los dos es mejor, pero es importante tener en cuenta que **jugador 1 no es necesariamente mejor**.

#### 3.4.2. Elección y comparación
***
Una vez determinadas las métricas, se realizaron las siguientes gráficas para comparar utilizando la cantidad de partidas ganadas:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia1.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Partidas ganadas para cada modelo entrenado de la instancia 1</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/instancia2.png" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Partidas ganadas para cada modelo entrenado de la instancia 2</label>
    </div>
</div>

**Observaciones generales:**
- La **instancia 2** alcanzó rendimientos más altos que la **instancia 1**
- Observando la gráfica, los modelos de la **instancia 1** se encuentran en condiciones parejas, excepto por aquellos entrenados utilizando $\gamma = 0.5$.
- Observando la gráfica, los "mejores" modelos de la **instancia 2** son aquellos con un peor ratio de victorias en la tabla 2 (compensado por su cantidad de partidas ganadas). Se determina que el peso de la cantidad de partidas ganadas es mayor al del ratio de victorias en este caso, ya que los otros ejemplos presentan una gran cantidad de empates.

**Interpretaciones:**
- La **instancia 1** es más robusta en relación a las distintas configuraciones paramétricas. La **instancia 2** en cambio es más sensible, devolviendo mejores resultados solo para el vector inicial con pesos pequeños. Esto puede implicar que si se entrenase más tiempo ambas instancias, la **instancia 1** alcance un mejor rendimiento que la **instancia 2**.
- Descartando el punto anterior, se interpreta que la separación de atributos sin normalizar (**instancia 2**) tiene un mejor rendimiento. Esto puede deberse a la pérdida de información al agrupar $A_i, B_i, C_i, D_i$ en $A, B, C, D$, al hecho de haber utilizado **normalización mín-máx** en los pesos, lo cual genera que uno sea siempre 0 y otro siempre 1, o a alguna otra razón no considerada.
- Tomando el hecho de que el vector con mejores resultados en la **instancia 2** es $w = [0.1, -0.9, 0.9, -0.1, 0.1, 0.1, -0.1, -0.1, 0.1]$, se puede interpretar que priorizar $A_1$ y $A_2$ por sobre el resto de pesos es una práctica acertada para este problema.

#### 3.4.4. Script 2
***
A continuación se adjunta un script que permite jugar una partida contra el jugador entrenado de mayor rendimiento.


In [1]:
# Script 2 - Partida VS mejor jugador entrenado
import ntPlay

Box(children=(Label(value='Representación del tablero:', layout=Layout(margin='0px 16px 0px 0px')), ToggleButt…


                                (   O   )

                            (   O   )(   O   )

                        (   O   )(   O   )(   O   )

                    (   O   )(   O   )(   O   )(   O   )

                (       )(       )(       )(       )(       )

            (       )(       )(       )(       )(       )(       )

        (       )(       )(       )(       )(       )(       )(       )

    (       )(       )(       )(       )(       )(       )(       )(       )

(       )(       )(       )(       )(       )(       )(       )(       )(       )

    (       )(       )(       )(       )(       )(       )(       )(       )

        (       )(       )(       )(       )(       )(       )(       )

            (       )(       )(       )(       )(       )(       )

                (       )(       )(       )(       )(       )

                    (   O   )(   O   )(   O   )(   O   )

                        (   O   )(   O   )(   O   )

                            (   O   )( 

Box(children=(Box(children=(Label(value='Movimiento de jugador 2:', layout=Layout(margin='0px 16px 0px 0px')),…

### 3.5. Entrenamiento contra aleatorio
***

El motivo de separar el entrenamiento del jugador aleatorio respecto al jugador previo, tiene su motivación en un hecho importante. Debido a las reglas de las **damas chinas**, si el oponente deja una pieza en su triángulo (un evento altamente probable considerando un jugador aleatorio) y el jugador comienza a llenar el triángulo opuesto (atascando la pieza del oponente), el juego no finalizará.

Para evitar esto, el jugador debería retirar piezas del triángulo del oponente, con el fin de habilitar movimientos que incentiven al oponente a retirar sus piezas del triangulo y que el jugador pueda colocar las suyas. Esto no sólo es contraintuitivo para el ajuste de los pesos propuestos para utilizar contra un jugador que esté intentando ganar, sino que puede demorar una cantidad descomunal de turnos.

Con el objetivo de evitar dicha demora, fue necesario implementar de nuevo el ajuste en empates, ya que en caso de no hacerlo, el modelo tardaría demasiado tiempo en poder realizar un avance. 

Este efecto puede apreciarse en las siguientes graficas generadas durante el entrenamiento de un jugador de la **instancia 1**:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/resultsRandom1.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.17 - Resultados de partidas en entrenamiento contra random</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/errorsRandom1.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.18 - Error cuadrático medio cada 10 iteraciones en entrenamiento contra random</label>
    </div>
</div>

De 100 iteraciones, las 100 partidas resultaron en empate. Además, el error cuadrático medio oscila violentamente y no disminuye dicha oscilación a medida que se ajusta el modelo. Esto puede deberse al hecho de que se contaron los empates para ajustar el modelo y como ya se mencionó anteriormente, esto introduce ruido al algoritmo de aprendizaje.

Al contrario de lo que se esperaba, al entrenar un jugador de la **instancia 2** se obtuvieron mejores resultados, pudiendo apreciarse esto en las siguientes gráficas:

<div style="margin-top: 16px; margin-bottom: 16px;">
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/resultsRandom2.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.19 - Resultados de partidas en entrenamiento contra random</label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/errorsRandom2.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 3.20 - Error cuadrático medio cada 10 iteraciones en entrenamiento contra random</label>
    </div>
</div>

De 100 iteraciones, el jugador logró ganar 27 partidas, empatando el resto. A su vez, el error cuadrático medio presentó una oscilación casi nula, con algunos picos. De este hecho puede observarse que la valoración de empates afecta menos al modelo de **instancia 2** que al de **instancia 1**, adaptandose a un jugador aleatorio y logrando ganarle en ciertos escenarios.


## 4. Conclusiones
***

#### 4.1. Respecto a los Oponentes
***
Observando los resultados a través de las iteraciones, se concluye que los dos oponentes con los que fue realizado el entrenamiento (jugador aleatorio y jugador previo) impactan de formas negativas al resultado final. Esto implica que la solución elegida es probable que no sea la adecuada para enfocar este tipo de problemas.

Para el caso del **oponente aleatorio**, el impacto negativo en el aprendizaje es causa de que se deben de tener en cuenta los empates, causando los mismos ruido en los ajustes y por tanto empeorando el aprendizaje.

Para el caso del **oponente previo**, si bien hay resultados relativamente buenos, es importante destacar como aprender meramente contra una versión previa de uno mismo genera un sesgo importante en el aprendizaje y produce _overfitting_. Los resultados finales buenos no necesariamente aseguran que el modelo entrenado vaya a tener resultados igual de buenos contra un jugador humano (o inclusive uno aleatorio con un comportamiento muy erratico).

#### 4.2. Respecto a los Atributos
***
Tanto empírica como estadísticamente se observa que la selección de atributos, en general, dió muy buenos resultados al inicio de los entrenamientos (es decir, con sus pesos iniciales). No fue hasta luego de varios ajustes que la eficacia comenzó a declinar. Esto habla muy bien de los atributos elegidos desde la perspectiva de un algoritmo greedy en un juego contra si mismo, dado que con múltiples pesos iniciales, el algoritmo dió buenos resultados hasta tomar una mala decisión en el ajuste.

Es importante destacar que esto se debe en gran parte a la inherente ventaja en las damas chinas que tiene el jugador que arranque primero. Es por esto que si se enfrentan dos jugadores muy parecidos a nivel de habilidad y estilo de juego, el que empiece tiene una ventaja y por ende mayor probabilidad de ganar.

Otra conclusión importante en relación a los atributos es que no siempre menos es más. Se consideró que agrupar pares de atributos en uno originalmente iba a dar mejores resultados pero en la experiencia no fue así. Esto no implica que no haya que agrupar nunca, sino que debe tenerse en cuenta ante cada escenario.

#### 4.3. Respecto a los Parámetros
***
Gracias a haber entrenado y estudiado todas las posibles combinaciones de ratio de aprendizaje y pesos iniciales se pudo encontrar modelos con un rendimiento relativamente bueno. Se concluye que estos parámetros tienen una influencia importante en el algoritmo de aprendizaje.

#### 4.4. Respecto a los Resultados
***
Debido a la naturaleza del juego, fue muy complejo determinar la eficacia de los jugadores entrenados. El tener que limitar la cantidad de turnos por partida para hacer viable el entrenamiento introdujo mucho ruido en el ajuste, afectando directamente al modelo.

#### 4.5. Posibles mejoras
***
Tomando en cuenta el actual modelo para resolver el problema planteado, se presentan ciertas estrategias para mejorar el rendimiento:
- Entrenar en una mayor cantidad de iteraciones, para ver si se alcanza una mejor convergencia
- Optimizar tiempos de ejecución, con el objetivo de poder entrenar más durante el mismo tiempo y poder aumentar el umbral de empate.
- Eliminar por completo el comportamiento de bucles por repetición de movimientos, buscando una forma de penalizar al jugador si lo hace.
- Normalizar los pesos con norma euclídea en vez de norma mín-máx, con el fin de no cancelar el peso de menor orden.
- Generar más instancias de prueba en la configuración paramétrica para poder aplicar un test de Friedman y un ad-hoc del mismo, con el fin de poder garantizar con más seguridad la mejor configuración paramétrica.